In [28]:
import pandas as pd
import numpy as np

In [42]:
def calculate_metrics(data):
    
    if 'timestamp' in data.columns:
        data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')  
    else:
        print("Warning: 'timestamp' column is missing. Please check the data.")

    
    data.dropna(inplace=True)

   
    data['position'] = data['side'] + '_' + data['positionSide']
    grouped = data.groupby('Port_ID')

    
    results = []

    for port_id, group in grouped:
        
        total_investment = group['quantity'].sum()
        net_profit = group['realizedProfit'].sum()
        positive_trades = group[group['realizedProfit'] > 0]
        win_positions = len(positive_trades)
        total_positions = len(group)
        
        
        roi = (net_profit / total_investment) * 100 if total_investment > 0 else 0
        win_rate = (win_positions / total_positions) * 100 if total_positions > 0 else 0
        
        
        daily_returns = group['realizedProfit'] / group['quantity']
        mean_return = daily_returns.mean()
        std_dev_return = daily_returns.std()
        sharpe_ratio = (mean_return / std_dev_return) if std_dev_return > 0 else 0
        
        
        cumulative_profit = group['realizedProfit'].cumsum()
        rolling_max = cumulative_profit.cummax()
        drawdown = rolling_max - cumulative_profit
        max_drawdown = drawdown.max()

        
        results.append({
            'Port_ID': port_id,
            'ROI': roi,
            'PnL': net_profit,
            'Sharpe_Ratio': sharpe_ratio,
            'MDD': max_drawdown,
            'Win_Rate': win_rate,
            'Win_Positions': win_positions,
            'Total_Positions': total_positions
        })

    
    results_df = pd.DataFrame(results)

    
    results_df['Score'] = (
        results_df['ROI'] * 0.4 +
        results_df['Sharpe_Ratio'] * 0.3 +
        results_df['Win_Rate'] * 0.2 -
        results_df['MDD'] * 0.1
    )
    results_df = results_df.sort_values(by='Score', ascending=False)

    return results_df


In [43]:

data = pd.read_csv('trade.csv')


In [44]:

print(data.columns)



Index(['Port_IDs', 'Trade_History'], dtype='object')


In [45]:


import ast


data = data.dropna(subset=['Trade_History'])


def safe_eval(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return None

data['Trade_History'] = data['Trade_History'].apply(safe_eval)


data = data.dropna(subset=['Trade_History'])


trade_history = data.explode('Trade_History')  
trade_history = pd.json_normalize(trade_history['Trade_History'])


print(trade_history.head())
print(trade_history.columns)


if 'timestamp' in trade_history.columns:
    trade_history['timestamp'] = pd.to_datetime(trade_history['timestamp'], errors='coerce')
else:
    print("No 'timestamp' column found. Please check your data structure.")


trade_history['Port_ID'] = data['Port_IDs'].repeat(data['Trade_History'].str.len()).values


metrics_df = calculate_metrics(trade_history)


print(metrics_df.head())


            time    symbol side      price       fee feeAsset    quantity  \
0  1718899656000   SOLUSDT  BUY  132.53700 -0.994027     USDT  1988.05500   
1  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796     USDT  1398.98088   
2  1718899618000  DOGEUSDT  BUY    0.12182 -0.039494     USDT   197.47022   
3  1718899616000  DOGEUSDT  BUY    0.12182 -0.008284     USDT    16.56752   
4  1718899616000  DOGEUSDT  BUY    0.12182 -0.046109     USDT    92.21774   

  quantityAsset  realizedProfit realizedProfitAsset baseAsset      qty  \
0          USDT             0.0                USDT       SOL     15.0   
1          USDT             0.0                USDT      DOGE  11484.0   
2          USDT             0.0                USDT      DOGE   1621.0   
3          USDT             0.0                USDT      DOGE    136.0   
4          USDT             0.0                USDT      DOGE    757.0   

  positionSide  activeBuy  
0         LONG       True  
1         LONG      False  
2       

In [47]:
metrics_df.to_csv('Calculated_Metrics.csv', index=False)


In [48]:
top_20 = metrics_df.head(20)
print(top_20)


                 Port_ID        ROI           PnL  Sharpe_Ratio        MDD  \
62   3986814617275053313   0.871628  16337.461881      1.288552   2.381947   
8    3826087012661391104  12.178517    532.656974      0.791784  12.925980   
47   3956048468100538880   1.114561   1373.564890      1.322352   0.000000   
16   3891020560590657281   1.743407   2856.300564      1.128399   0.000000   
2    3768170840939476993   8.779089    243.668899      0.834517   0.000000   
144  4039279455324236544   1.022101   1038.807419      1.101072   0.000000   
92   4017110277719148289   0.471231   2899.933021      0.796681   0.000000   
14   3886752488982104320   0.899440   7195.178325      0.583919  38.208100   
94   4017323550554338817   1.957286   3134.735103      0.592944   0.000000   
133  4035430878731345664   1.909902   2493.750420      0.947716   0.000000   
143  4039129759104249600   3.162479   1264.289200      0.886581   0.000000   
74   3998659472131949824   0.307402   1449.325940      0.558916 